In [ ]:
#default_exp data

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
#export
import torch, random, pandas as pd, os, warnings, shutil, uuid
from torch.utils.data import DataLoader, RandomSampler
from datasets import load_dataset, load_from_disk, DatasetDict, ClassLabel
from IPython.display import display, HTML
from travis_attack.models import get_vm_probs
from travis_attack.config import Config
from travis_attack.utils import robust_rmtree

In [ ]:
#hide
import inspect
from IPython.core.debugger import set_trace

# Preparing data

## Classes 

### Base class

In [ ]:
#export
class ProcessedDataset: 
    """Class that wraps a raw dataset (e.g. from huggingface datasets) and performs preprocessing on it."""
    def __init__(self, cfg, vm_tokenizer, vm_model, pp_tokenizer, sts_model,
                 load_processed_from_file=True): 
        """load_processed_from_file: set to true to load completed version from file, false will process the data. """
        self._cfg,self._vm_tokenizer,self._vm_model,self._pp_tokenizer,self._sts_model = cfg,vm_tokenizer,vm_model,pp_tokenizer,sts_model
        shard_suffix = f"_{self._cfg.n_shards}_shards" if self._cfg.use_small_ds else ""
        self.cache_path_raw = f"{self._cfg.path_data_cache}{self._cfg.dataset_name}_raw{shard_suffix}"
        self.cache_path_tkn = f"{self._cfg.path_data_cache}{self._cfg.dataset_name}_tkn{shard_suffix}"
        
        if load_processed_from_file:
            if os.path.exists(self.cache_path_raw) and os.path.exists(self.cache_path_tkn): 
                self.dsd_raw = load_from_disk(self.cache_path_raw) 
                self.dsd_tkn = load_from_disk(self.cache_path_tkn)
                self._prep_dataloaders()
            else: 
                warnings.warn("Cache file not found, so will now process the raw dataset.")
                self._preprocess_dataset() 
        else:   
            self._preprocess_dataset() 
        self._update_cfg()
            
    def _prep_dsd_raw_simple(self): 
        """Load the simple dataset and package it up in a DatasetDict (dsd) 
        with splits for train, valid, test."""
        self.dsd_raw = DatasetDict()
        for s in self._cfg.splits:  
            self.dsd_raw[s] = load_dataset('csv', 
                data_files=f"{self._cfg.path_data}simple_dataset_{s}.csv", keep_in_memory=False)['train']    
        
    def _prep_dsd_raw_rotten_tomatoes(self):
        """Load the rotten tomatoes dataet and package it up in a DatasetDict (dsd) 
        with splits for train, valid, test."""
        self.dsd_raw = load_dataset("rotten_tomatoes")
        self.dsd_raw['valid'] = self.dsd_raw.pop('validation')  # "valid" is easier than "validation" 
        # make sure that all datasets have the same number of labels as what the victim model predicts
        for _,ds in self.dsd_raw.items(): assert ds.features[self._cfg.label_cname].num_classes == self._cfg.vm_num_labels 

    def _prep_dsd_raw_snli(self): 
        ## For snli
        # remove_minus1_labels = lambda x: x[label_cname] != -1
        # ds_train = ds_train.filter(remove_minus1_labels)
        # valid = valid.filter(remove_minus1_labels)
        # test = test.filter(remove_minus1_labels)
        raise NotImplementedError("SNLI not implemented yet.")
    
    def _preprocess_dataset(self): 
        """Add columns, tokenize, transform, prepare dataloaders, and do other preprocessing tasks."""
        ##### TODO: why do you need train_eval? how is it different from train?
        if   self._cfg.dataset_name == "simple":          self._prep_dsd_raw_simple()
        elif self._cfg.dataset_name == "rotten_tomatoes": self._prep_dsd_raw_rotten_tomatoes()
        else: raise Exception("cfg.dataset_name must be either 'simple' or 'rotten_tomatoes'")
            
        self.dsd_raw = self.dsd_raw.map(self._add_idx, batched=True, with_indices=True)  # add idx column
        if self._cfg.use_small_ds: self._shard_dsd_raw()  # do after adding idx so it's consistent across runs
        # add VM score & filter out misclassified examples. (at this point lengths are diff between dsd_raw and dsd_tkn)
        self.dsd_tkn = self.dsd_raw.map(self._add_vm_orig_score, batched=True)  
        if self._cfg.remove_misclassified_examples: 
            self.dsd_tkn = self.dsd_tkn.filter(lambda x: x['orig_vm_predclass'] == x['label']) 
        self.dsd_tkn = self.dsd_tkn.map(self._add_sts_orig_embeddings, batched=True)  # add STS score 
        self.dsd_tkn = self.dsd_tkn.map(self._tokenize_fn,             batched=True)  # tokenize
        self.dsd_tkn = self.dsd_tkn.map(self._add_n_tokens,            batched=True)  # add n_tokens
        if self._cfg.bucket_by_length: self.dsd_tkn = self.dsd_tkn.sort("n_tokens", reverse=True)  # sort by n_tokens (high to low), useful for cuda memory caching
        
        # filter out rows in dsd_raw that aren't in dsd_tkn
        for s in self._cfg.splits:
            idx_list = self.dsd_tkn[s]['idx']
            self.dsd_raw[s] = self.dsd_raw[s].filter(lambda x: x['idx'] in idx_list)
        for s in self._cfg.splits: assert len(self.dsd_raw[s]) == len(self.dsd_tkn[s])  # check ds has same number of elements in raw and tkn
        
        # save to cache 
        self._cache_processed_ds()
        self._prep_dataloaders()
        
    def _prep_dataloaders(self): 
        self.dld_raw = self._get_dataloaders_dict(self.dsd_raw, collate_fn=self._collate_fn_raw)  # dict of data loaders that serve raw text
        self.dld_tkn = self._get_dataloaders_dict(self.dsd_tkn, collate_fn=self._collate_fn_tkn)  # dict of data loaders that serve tokenized text
        
    def _add_idx(self, batch, idx):
        """Add row numbers"""
        batch['idx'] = idx 
        return batch   
    
    def _add_n_tokens(self, batch): 
        """Add the number of tokens of the orig text """
        batch['n_tokens'] = [len(o) for o in batch['input_ids']]
        return batch 
    
    def _add_sts_orig_embeddings(self, batch): 
        """Add the sts embeddings of the orig text"""
        batch['orig_sts_embeddings'] = self._sts_model.encode(batch[self._cfg.orig_cname], batch_size=64, convert_to_tensor=False)
        return batch
    
    def _add_vm_orig_score(self, batch): 
        """Add the vm score of the orig text"""
        labels = torch.tensor(batch[self._cfg.label_cname], device=self._cfg.device)
        orig_probs,orig_predclass = get_vm_probs(batch[self._cfg.orig_cname], self._cfg, self._vm_tokenizer,
                                                 self._vm_model, return_predclass=True)
        batch['orig_truelabel_probs'] = torch.gather(orig_probs,1, labels[:,None]).squeeze().cpu().tolist()
        batch['orig_vm_predclass'] = orig_predclass.cpu().tolist()
        return batch
    
    def _tokenize_fn(self, batch):  
        """Tokenize a batch of orig text using the paraphrase tokenizer."""
        return self._pp_tokenizer(batch[self._cfg.orig_cname], truncation=True, max_length=self._cfg.orig_max_length)  
    
    def _collate_fn_tkn(self, x): 
        """Collate function used by the DataLoader that serves tokenized data."""
        d = dict()
        for k in ['idx', 'attention_mask', 'input_ids', 'label', 'orig_truelabel_probs', 'orig_sts_embeddings']: 
            d[k] = [o[k] for o in x]
        return self._pp_tokenizer.pad(d, pad_to_multiple_of=self._cfg.orig_padding_multiple, return_tensors="pt")

    def _collate_fn_raw(self, x): 
        """Collate function used by the DataLoader that serves raw data."""
        d = dict()
        for k in [self._cfg.orig_cname, 'idx']: d[k] = [o[k] for o in x]
        return d 

    def _get_sampler(self, ds): 
        """Returns a RandomSampler. Used so we can keep the same shuffle order across multiple data loaders.
        Used when self._cfg.shuffle_train = True"""
        g = torch.Generator()
        g.manual_seed(seed)
        return RandomSampler(ds, generator=g)
    
    def _shard_dsd_raw(self):
        """Replaces dsd_raw with a smaller shard of itself."""
        for k,v in self.dsd_raw.items():  
            self.dsd_raw[k] = v.shard(self._cfg.n_shards, 0, contiguous=self._cfg.shard_contiguous)
        
    def _get_dataloaders_dict(self, dsd, collate_fn): 
        """Prepare a dict of dataloaders for train, valid and test"""
        if self._cfg.bucket_by_length and self._cfg.shuffle_train:  raise Exception("Can only do one of bucket by length or shuffle")
        d = dict()
        for split, ds in dsd.items(): 
            if self._cfg.shuffle_train:
                if split == "train": 
                    sampler = self.get_sampler(ds)
                    d[split] =  DataLoader(ds, batch_size=self._cfg.batch_size_train, 
                                           sampler=sampler, collate_fn=collate_fn, 
                                           num_workers=self._cfg.n_wkrs, pin_memory=self._cfg.pin_memory) 
                else: 
                    d[split] =  DataLoader(ds, batch_size=self._cfg.batch_size_eval, 
                                           shuffle=False, collate_fn=collate_fn, 
                                           num_workers=self._cfg.n_wkrs, pin_memory=self._cfg.pin_memory) 
            if self._cfg.bucket_by_length: 
                batch_size = self._cfg.batch_size_train if split == "train" else self._cfg.batch_size_eval
                d[split] =  DataLoader(ds, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, 
                                       num_workers=self._cfg.n_wkrs, pin_memory=self._cfg.pin_memory) 

        # Add eval dataloader for train 
        d['train_eval'] = DataLoader(dsd['train'], batch_size=self._cfg.batch_size_eval, shuffle=False,
                                    collate_fn=collate_fn, 
                                     num_workers=self._cfg.n_wkrs, pin_memory=self._cfg.pin_memory) 
        return d 
    
    def _update_cfg(self): 
        self._cfg.ds_length = dict()
        
        for k in self.dsd_raw.keys(): 
            self._cfg.ds_length[k] = len(self.dsd_raw[k])
            
    def _cache_processed_ds(self):
        def _reset_dir(path): 
            if os.path.exists(path) and os.path.isdir(path):    
#                robust_rmtree(path, logger=None, max_retries=6)  
                # So deleting the old files sometimes throws errors because of race conditions, I think 
                # so as a workaround we will just move files to old directories and then periodicallly clean them. 
                path_old_files = f"{self._cfg.path_data_cache}old_files/"
                os.makedirs(path_old_files, exist_ok=True)
                shutil.move(path, f"{path_old_files}{uuid.uuid4().hex}") 
            os.makedirs(path, exist_ok=True)
        _reset_dir(self.cache_path_raw)
        _reset_dir(self.cache_path_tkn)
        self.dsd_raw.save_to_disk(dataset_dict_path = self.cache_path_raw)
        self.dsd_tkn.save_to_disk(dataset_dict_path = self.cache_path_tkn)
        
    def show_random_elements(self, ds, num_examples=10):
        """Print some elements in a nice format so you can take a look at them. 
        Split is one of 'train', 'test', 'valid'. 
        Use for a dataset `ds` from the `dataset` package.  """
        assert num_examples <= len(ds), "Can't pick more elements than there are in the dataset."
        picks = []
        for _ in range(num_examples):
            pick = random.randint(0, len(ds)-1)
            while pick in picks:
                pick = random.randint(0, len(ds)-1)
            picks.append(pick)
        df = pd.DataFrame(ds[picks])
        for column, typ in ds.features.items():
            if isinstance(typ, ClassLabel):
                df[column] = df[column].transform(lambda i: typ.names[i])
        display(HTML(df.to_html()))

## Usage 

### Basics 

Here we have defined a class `ProcessedDataset` that will load and preprocess a dataset. But before processing the dataset you must load both the config object and all models/tokenizers, so we do this first. 

In [ ]:
from travis_attack.models import prepare_models
cfg = Config()
vm_tokenizer, vm_model, pp_tokenizer, pp_model, sts_model, cfg = prepare_models(cfg)

Currently there are two choices for dataset: 

* `simple`, a dataset of simple sentences with four elements each in the train, test and valid splits
* `rotten_tomatoes`, a dataset of movie reviews scraped from the Rotten Tomatoes site. 

The dataset is specified by the config class. There are two ways to do this. 

1. Edit the `self.dataset_name` variable in the Config class to either `simple` or `rotten_tomatoes`. An error will be thrown if the name is not one of these two. This is the best way to use when doing runs.   
2. Use the `adjust_dataset_...` methods of the config class: e.g. `cfg = cfg.adjust_dataset_for_rotten_tomatoes_dataset() = Config()`. This is easiest for automated testing so we will do this here. 

Once the config is specified and loaded, create an object of class `ProcessedDataset` by passing the config, models and tokenizers as variables. This will do all preprocessing automatically in creating the object (the preprocessing code is in the `__init__()` function of the class.  

In [ ]:
cfg_simple = cfg.adjust_config_for_simple_dataset()
ds = ProcessedDataset(cfg_simple, vm_tokenizer, vm_model, pp_tokenizer, sts_model, 
                      load_processed_from_file=False)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

If you want to use the small dataset adjust the config before creating the `ProcessedDataset` object. 

In [ ]:
cfg_rt_small_ds = cfg.adjust_config_for_rotten_tomatoes_dataset().small_ds()
ds = ProcessedDataset(cfg_rt_small_ds, vm_tokenizer, vm_model, pp_tokenizer, sts_model,
                      load_processed_from_file=False)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

### Accessing datasets

You can access raw data with `ds.dsd_raw` and processed data with the `ds.dsd_tkn`. (The dsd here stands for "DatasetDict")

In [ ]:
ds.dsd_raw

In [ ]:
ds.dsd_tkn

You can access elements by indexing: 

In [ ]:
ds.dsd_raw['valid'][0:2]

In [ ]:
ds.dsd_tkn['valid'][0:2]

Alternately you can look at some random elements of a dataset with the `ds.show_random_elements()` method. 

In [ ]:
ds.show_random_elements(ds.dsd_raw['train'], num_examples=3)

In [ ]:
ds.show_random_elements(ds.dsd_tkn['train'], num_examples=3)

### Accessing DataLoaders 

We can access dataloaders for the raw text in `ds.dsd_raw` with `ds.dld_raw`, and for the tokenised text in `ds.dsd_tkn` with `ds_dld_tkn`. Both of these are dictionaries of dataloaders with keys `['train', 'valid', 'test', 'train_eval]`. 

In [ ]:
print("Dataloader dict has keys:", ds.dld_raw.keys())
batch_raw = next(iter(ds.dld_raw['train']))
batch_tkn = next(iter(ds.dld_tkn['train']))
print(batch_raw.keys())
print(batch_tkn.keys())
print("Tokenised input is of shape:", batch_tkn['input_ids'].shape)

#hide
## Export 

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()